In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [15]:
from datasets import load_dataset, Dataset

In [16]:
dataset = load_dataset('Vampyrian/products_with_category_household_equipments')
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 26056
    })
    validation: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 3453
    })
    test: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 3453
    })
})

In [20]:
dataset['train'][3]

{'text': 'BEKO Refrigerator TSE1423N 84 cm, Energy class E (old A++), White',
 'label': 0,
 'label_text': 'Šaldytuvai'}

In [30]:
unique_categories = set(dataset['train']['label_text'])
id2label = {idx: label for idx, label in enumerate(unique_categories)}
label2id = {label: idx for idx, label in enumerate(unique_categories)}

print(len(id2label))
print('----')
print("id2label:", id2label)
print('----')
print("label2id:", label2id)

79
----
id2label: {0: 'Sumuštinių keptuvės', 1: 'Garų puodai', 2: 'Dulkių siurbliai', 3: 'Virduliai', 4: 'Maisto atliekų smulkintuvai', 5: 'Sulčiaspaudės', 6: 'Rankiniai dulkių siurbliai', 7: 'Šilumos siurbliai oras-oras', 8: 'Plaukų džiovintuvai', 9: 'Epiliatoriai', 10: 'Gartraukių filtrai', 11: 'Platforminės svarstyklės', 12: 'Oro kondicionieriai', 13: 'Kaitlentės', 14: 'Skrudintuvai', 15: 'Pūkų rinkikliai', 16: 'Konvekciniai radiatoriai', 17: 'Infraraudonųjų spindulių šildytuvai', 18: 'Skalbyklės - džiovyklės', 19: 'Vakuumatoriai', 20: 'Cikloninės krosnelės', 21: 'Kiaušinių virimo aparatai', 22: 'Langų valytuvai', 23: 'Skalbimo mašinos', 24: 'Oro šildytuvai', 25: 'Oro drėkintuvai', 26: 'Kavos aparatai', 27: 'Elektriniai šildytuvai', 28: 'Barzdaskutės', 29: 'Plaukų kirpimo mašinėlės', 30: 'Dulkių siurbliai robotai', 31: 'Ventiliatoriai', 32: 'Lygintuvai', 33: 'Blenderiai', 34: 'Viryklės', 35: 'Įmontuojamos indaplovės', 36: 'Griliai ir keptuvės', 37: 'Nešiojami oro kondicionieriai', 3

In [5]:
def replace_category_with_id(example):
    example['label_id'] = label2id[example.pop('label_text')]
    return example

dataset = dataset.map(replace_category_with_id)


Map:   0%|          | 0/26056 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [6]:
dataset['train'][1]

{'text': 'Electrolux LRC5ME38W2 -jääkaappi, valkoinen',
 'label': 0,
 'label_id': 77}

# Tokenizing

In [21]:
checkpoint = 'google-bert/bert-base-multilingual-cased'
checkpoint = 'amberoad/bert-multilingual-passage-reranking-msmarco'

In [22]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/Users/zilvinasmacius/PycharmProjects/kategorijos_huggin_face/.venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [23]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [26]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/26056 [00:00<?, ? examples/s]

In [29]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [11]:
import evaluate
accuracy = evaluate.load("accuracy")

In [12]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

# Training started....

In [28]:
from transformers import TrainingArguments, Trainer, AutoModelForSequenceClassification

In [32]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=len(id2label), ignore_mismatched_sizes=True)

training_args = TrainingArguments(output_dir="test_trainer", eval_strategy="epoch")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at amberoad/bert-multilingual-passage-reranking-msmarco and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([2, 768]) in the checkpoint and torch.Size([79, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([2]) in the checkpoint and torch.Size([79]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [33]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
[codecarbon INFO @ 15:50:52] [setup] RAM Tracking...
[codecarbon INFO @ 15:50:52] [setup] GPU Tracking...
[codecarbon INFO @ 15:50:52] No GPU found.
[codecarbon INFO @ 15:50:52] [setup] CPU Tracking...
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To d

In [34]:
trainer.train()

ValueError: expected sequence of length 21 at dim 1 (got 24)

In [13]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

In [30]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=len(id2label), id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at amberoad/bert-multilingual-passage-reranking-msmarco and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([2, 768]) in the checkpoint and torch.Size([79, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([2]) in the checkpoint and torch.Size([79]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
training_args = TrainingArguments(
    output_dir="category_predictor_for_household_equipments",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_total_limit=2,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/Users/zilvinasmacius/PycharmProjects/kategorijos_huggin_face/.venv/lib/python3.12/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
[codecarbon INFO @ 13:24:46] [setup] RAM Tracking...
[codecarbon INFO @ 13:24:46] [setup] GPU Tracking...
[codecarbon INFO @ 13:24:46] No GPU found.
[codecarbon INFO @ 13:24:46] [setup] CPU Tracking...
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- 

In [36]:
# import torch
# torch.cuda.empty_cache() # Atminčiai sumazinti

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.273200,0.982593,0.783956
2,0.741200,0.689020,0.836085
3,0.536500,0.613182,0.849986


[codecarbon INFO @ 13:25:22] Energy consumed for RAM : 0.000025 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 13:25:22] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 13:25:22] 0.000202 kWh of electricity used since the beginning.
[codecarbon INFO @ 13:25:37] Energy consumed for RAM : 0.000050 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 13:25:37] Energy consumed for all CPUs : 0.000354 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 13:25:37] 0.000404 kWh of electricity used since the beginning.
[codecarbon INFO @ 13:25:52] Energy consumed for RAM : 0.000075 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 13:25:52] Energy consumed for all CPUs : 0.000531 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 13:25:52] 0.000606 kWh of electricity used since the beginning.
[codecarbon INFO @ 13:26:07] Energy consumed for RAM : 0.000100 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 13:26:07] Energy consumed for all CPUs : 0.000709 kWh. Total CPU Power : 42.5 W
[codeca

TrainOutput(global_step=4887, training_loss=1.1745200784557164, metrics={'train_runtime': 1904.2071, 'train_samples_per_second': 41.05, 'train_steps_per_second': 2.566, 'total_flos': 1118316268051440.0, 'train_loss': 1.1745200784557164, 'epoch': 3.0})

In [1]:
from transformers import pipeline

classifier = pipeline(task="text-classification", model="Vampyrian/category_predictor_for_household_equipments")

OSError: Can't load tokenizer for 'Vampyrian/category_predictor_for_household_equipments'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'Vampyrian/category_predictor_for_household_equipments' is the correct path to a directory containing all relevant files for a BertTokenizerFast tokenizer.

In [12]:
text = "visas tarpinių komplektas, variklis ENT000054"
prediction = classifier(text)
print(prediction)

[{'label': 'Griliai ir keptuvės', 'score': 0.3706127405166626}]


# AUTOTRAIN